<h1> Подраздел 1.5 <span class="tocSkip"></span></h1>

<h1> Задание <span class="tocSkip"></span></h1>

Требуется перевести спарсенный json в формат csv. 

А также сгруппировать вакансии по направлениями (DS, DE, Software Engenering, etc.) и ответить на вопросы:

1. Какая средняя и медианная зарплата по группам вакансий?
2. Какая средняя и медианная зарплата по каждому региону?
3. Какая самая высокооплачиваемая из групп вакансий, исходя из их средних зарплат?
4. Какое процентное соотношение каждого региона по вакансиям от всех вакансий?
5. Какая корреляция уровня опыта от зарплаты?
6. Сколько должностей в наборе данных?
7. Какие 10 наиболее часто встречающихся должностей?


## Загрузка и обработка данных

In [1]:
import json
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('ds_salaries.csv', index_col = 0)
print(df.info())
df.head()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 607 entries, 0 to 606
Data columns (total 11 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   work_year           607 non-null    int64 
 1   experience_level    607 non-null    object
 2   employment_type     607 non-null    object
 3   job_title           607 non-null    object
 4   salary              607 non-null    int64 
 5   salary_currency     607 non-null    object
 6   salary_in_usd       607 non-null    int64 
 7   employee_residence  607 non-null    object
 8   remote_ratio        607 non-null    int64 
 9   company_location    607 non-null    object
 10  company_size        607 non-null    object
dtypes: int64(4), object(7)
memory usage: 56.9+ KB
None


,work_year,experience_level,employment_type,job_title,salary,salary_currency,salary_in_usd,employee_residence,remote_ratio,company_location,company_size
0,2020,MI,FT,Data Scientist,70000,EUR,79833,DE,0,DE,L
1,2020,SE,FT,Machine Learning Scientist,260000,USD,260000,JP,0,JP,S
2,2020,SE,FT,Big Data Engineer,85000,GBP,109024,GB,50,GB,M
3,2020,MI,FT,Product Data Analyst,20000,USD,20000,HN,0,HN,S
4,2020,SE,FT,Machine Learning Engineer,150000,USD,150000,US,50,US,L


Видим, что пропусков вроде бы нет

Удалим дубликаты,если они есть, со сдигом индексов

In [3]:
df = df.drop_duplicates().reset_index(drop=True)
df

,work_year,experience_level,employment_type,job_title,salary,salary_currency,salary_in_usd,employee_residence,remote_ratio,company_location,company_size
0,2020,MI,FT,Data Scientist,70000,EUR,79833,DE,0,DE,L
1,2020,SE,FT,Machine Learning Scientist,260000,USD,260000,JP,0,JP,S
2,2020,SE,FT,Big Data Engineer,85000,GBP,109024,GB,50,GB,M
3,2020,MI,FT,Product Data Analyst,20000,USD,20000,HN,0,HN,S
4,2020,SE,FT,Machine Learning Engineer,150000,USD,150000,US,50,US,L
...,...,...,...,...,...,...,...,...,...,...,...
560,2022,SE,FT,Data Engineer,154000,USD,154000,US,100,US,M
561,2022,SE,FT,Data Engineer,126000,USD,126000,US,100,US,M
562,2022,SE,FT,Data Analyst,129000,USD,129000,US,0,US,M
563,2022,SE,FT,Data Analyst,150000,USD,150000,US,100,US,M


### Какая средняя и медианная зарплата по группам вакансий?

Средние зарплаты по должностям:

In [4]:
df.groupby('job_title')['salary'].mean()/1000

job_title
3D Computer Vision Researcher                400.000000
AI Scientist                                 290.571429
Analytics Engineer                           175.000000
Applied Data Scientist                       172.400000
Applied Machine Learning Scientist           141.350000
BI Data Analyst                             1902.045333
Big Data Architect                           125.000000
Big Data Engineer                            455.000000
Business Data Analyst                        355.000000
Cloud Data Engineer                          140.000000
Computer Vision Engineer                      83.500000
Computer Vision Software Engineer            100.333333
Data Analyst                                  94.480500
Data Analytics Engineer                       61.750000
Data Analytics Lead                          405.000000
Data Analytics Manager                       127.134286
Data Architect                               177.873909
Data Engineer                         

Медианные зарплаты по должностям:

In [5]:
df.groupby('job_title')['salary'].median()/1000

job_title
3D Computer Vision Researcher                400.0000
AI Scientist                                 120.0000
Analytics Engineer                           179.8500
Applied Data Scientist                       157.0000
Applied Machine Learning Scientist            56.7000
BI Data Analyst                               99.0000
Big Data Architect                           125.0000
Big Data Engineer                             92.5000
Business Data Analyst                        100.0000
Cloud Data Engineer                          140.0000
Computer Vision Engineer                      81.0000
Computer Vision Software Engineer             81.0000
Data Analyst                                  90.1600
Data Analytics Engineer                       58.5000
Data Analytics Lead                          405.0000
Data Analytics Manager                       120.0000
Data Architect                               180.0000
Data Engineer                                108.0000
Data Engineering M

### Какая самая высокооплачиваемая из групп вакансий, исходя из их средних зарплат?

Выведем 10 самых топовых вакансий

In [6]:
(df.groupby('job_title')['salary'].mean()/1000).sort_values(ascending = False).head(10)

job_title
Head of Machine Learning         6000.000000
ML Engineer                      2676.666667
BI Data Analyst                  1902.045333
Lead Data Scientist              1101.666667
Data Science Manager             1062.598583
Lead Data Analyst                 569.000000
Data Scientist                    543.619154
Big Data Engineer                 455.000000
Data Analytics Lead               405.000000
3D Computer Vision Researcher     400.000000
Name: salary, dtype: float64

Самая крутая вакансия:

In [7]:
(df.groupby('job_title')['salary'].mean()/1000).sort_values(ascending = False).head(1)

job_title
Head of Machine Learning    6000.0
Name: salary, dtype: float64

### Какая средняя и медианная зарплата по каждому региону?

Отсортированные средние зарплаты по региону:

In [8]:
(df.groupby('employee_residence')['salary'].mean()/1000).sort_values(ascending = False)

employee_residence
CL    30400.000000
HU    11000.000000
JP     2927.428571
IN     1870.166567
MX      389.000000
CN      299.000000
DK      240.000000
MY      200.000000
TR      179.333333
PR      160.000000
US      150.782254
SG      140.000000
PL      135.000000
AU      130.333333
NZ      125.000000
CH      115.000000
CA      113.885185
RU      105.750000
JE      100.000000
AE      100.000000
IQ      100.000000
DZ      100.000000
BG       80.000000
BR       77.933333
BO       75.000000
DE       73.539958
BE       72.500000
CZ       69.999000
AT       68.000000
IE       65.000000
GB       62.159442
AR       60.000000
NL       56.280000
SI       54.000000
IT       53.050000
ES       53.010667
FR       52.512222
GR       50.500000
LU       50.000000
HK       48.000000
PH       45.760000
RO       43.500000
HR       40.000000
PT       39.066667
VN       30.800000
EE       30.000000
NG       30.000000
TN       29.000000
PK       25.466667
MT       24.000000
CO       21.844000
RS       21.

Отсортированные медианные зарплаты по региону:

In [9]:
(df.groupby('employee_residence')['salary'].median()/1000).sort_values(ascending = False)

employee_residence
CL    30400.000
HU    11000.000
IN     1425.000
MX      389.000
CN      299.000
JP      260.000
DK      240.000
MY      200.000
TR      180.000
PR      160.000
SG      140.000
US      138.350
PL      130.000
NZ      125.000
AU      121.000
AE      115.000
CH      115.000
JE      100.000
IQ      100.000
DZ      100.000
CA      100.000
BR       84.800
BG       80.000
BO       75.000
BE       72.500
RU       72.500
CZ       69.999
DE       66.000
AT       65.000
IE       65.000
GB       60.000
AR       60.000
NL       57.000
SI       54.000
FR       50.000
LU       50.000
HK       48.000
PT       47.700
GR       47.500
PH       45.760
ES       45.000
RO       43.500
HR       40.000
VN       38.400
IT       31.600
NG       30.000
EE       30.000
TN       29.000
MT       24.000
CO       21.844
RS       21.600
HN       20.000
MD       18.000
PK       16.000
UA       13.400
KE        9.272
IR        4.000
Name: salary, dtype: float64

### Какое процентное соотношение каждого региона по вакансиям от всех вакансий?

Всего вакансий:

In [22]:
len(df)

565

Процентное соотношение каждого региона по вакансиям от всех вакансий:

In [33]:
print(round(df.groupby('employee_residence')['salary'].count()/len(df), 5))

employee_residence
AE    0.00531
AR    0.00177
AT    0.00531
AU    0.00531
BE    0.00354
BG    0.00177
BO    0.00177
BR    0.01062
CA    0.04779
CH    0.00177
CL    0.00177
CN    0.00177
CO    0.00177
CZ    0.00177
DE    0.04248
DK    0.00354
DZ    0.00177
EE    0.00177
ES    0.02655
FR    0.03186
GB    0.07611
GR    0.02124
HK    0.00177
HN    0.00177
HR    0.00177
HU    0.00354
IE    0.00177
IN    0.05310
IQ    0.00177
IR    0.00177
IT    0.00708
JE    0.00177
JP    0.01239
KE    0.00177
LU    0.00177
MD    0.00177
MT    0.00177
MX    0.00354
MY    0.00177
NG    0.00354
NL    0.00885
NZ    0.00177
PH    0.00177
PK    0.01062
PL    0.00708
PR    0.00177
PT    0.01062
RO    0.00354
RS    0.00177
RU    0.00708
SG    0.00354
SI    0.00354
TN    0.00177
TR    0.00531
UA    0.00177
US    0.52212
VN    0.00531
Name: salary, dtype: float64


### Какая корреляция уровня опыта от зарплаты?

Посмотрим уникальные значения в признаке experience_level

In [10]:
df.experience_level.value_counts()

SE    243
MI    208
EN     88
EX     26
Name: experience_level, dtype: int64

Создадим новый столбец, содержащий 4 различных категории уровня опыта

In [11]:
def create_experience_category(i):
    
    if i == 'SE':
        return 0
    if i == 'MI':
        return 1
    if i == 'EN':
        return 2
    if i == 'EX':
        return 3 
      

In [12]:
df['experience_category'] = df['experience_level'].apply(create_experience_category)

Посмотрим уникальные значения в столбце work_year

In [13]:
df.work_year.value_counts()

2022    278
2021    215
2020     72
Name: work_year, dtype: int64

Построим матрицу корреляции

In [14]:
corr = df[['experience_category', 'salary', 'work_year']].corr().style.background_gradient(cmap='coolwarm')
corr

,experience_category,salary,work_year
experience_category,1.000000,0.033029,-0.228958
salary,0.033029,1.000000,-0.082597
work_year,-0.228958,-0.082597,1.000000


Видим, что корреляция отсутствует.

### Сколько должностей в наборе данных?

Посмотрим уникальные названия должностей

In [15]:
print(len(df.job_title.value_counts()))
df.job_title.value_counts()

50


Data Scientist                              130
Data Engineer                               121
Data Analyst                                 82
Machine Learning Engineer                    39
Research Scientist                           16
Data Science Manager                         12
Data Architect                               11
Big Data Engineer                             8
Machine Learning Scientist                    8
Principal Data Scientist                      7
AI Scientist                                  7
Data Science Consultant                       7
Director of Data Science                      7
Data Analytics Manager                        7
ML Engineer                                   6
Computer Vision Engineer                      6
BI Data Analyst                               6
Lead Data Engineer                            6
Data Engineering Manager                      5
Business Data Analyst                         5
Head of Data                            

Всего 50 должностей

### Какие 10 наиболее часто встречающихся должностей?

Выведем 10 наиболее часто встречающихся должностей на экран

In [16]:
print(f'10 наиболее часто встречающихся должностей:\n {list(df.job_title.value_counts().index[:10])}')

10 наиболее часто встречающихся должностей:
 ['Data Scientist', 'Data Engineer', 'Data Analyst', 'Machine Learning Engineer', 'Research Scientist', 'Data Science Manager', 'Data Architect', 'Big Data Engineer', 'Machine Learning Scientist', 'Principal Data Scientist']
